<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/hypernets_surface_reflectance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Band integration for HYPERNETS**

This is an example of how the matheo tool (https://matheo.readthedocs.io/en/latest/) can be used to band integrate the HYPERNETS products over various satellite spectral response functions (SRF).

We first install the obsarray package (flag handling and accessing uncertainties), the punpy package (uncertainty propagation) and the matheo package (for band integration).

In [ ]:
!pip install matheo

Next, we open the HYPERNETS L2B data. An example for Gobabeb is used and available from the comet_training repository (which is first cloned). 

In [ ]:
!git clone https://github.com/comet-toolkit/comet_training.git

In [ ]:
import xarray as xr
import numpy as np
import obsarray

ds_HYP = xr.open_dataset("HYPERNETS_L_GHNA_L2A_REF_20231103T0901_20240124T2246_v2.0.nc")  # read digital effects table
reflectance_HYP=ds_HYP["reflectance"].values
u_reflectance_HYP=ds_HYP.unc["reflectance"].total_unc()  # use obsarray to get total uncertainties
wavelength_HYP=ds_HYP["wavelength"].values

As described in https://matheo.readthedocs.io/en/latest/content/user_guide.html, there are three main methods to perform band integration using matheo. 
The first way is using the platform name and sensor name for an earth observation sensor which is included in pyspectral (see https://pyspectral.readthedocs.io/en/master/platforms_supported.html for supported platforms), it is possible to just specify the platform and sensor as a string using the spectral_band_int_sensor() function. Here we perform the band integration over the S2A spectral response function:

In [ ]:
from matheo.band_integration import band_integration

refl_band, band_centres = band_integration.spectral_band_int_sensor(
    d=reflectance_HYP,
    wl=wavelength_HYP,
    platform_name="Sentinel-2A",
    sensor_name="MSI",
    u_d=None,
)
print(band_centres, refl_band)                          

Alternatively, for satellites not included in this list it is possible to use manually specified SRFs. One can either specify the widths of each band (here all bands have width of 10 nm, but variable width per band is also possible) and provide a band_shape (here triangular SRF is assumed, other options include tophat and Gaussian) using:

In [ ]:
wav_sat = np.arange(400,1600,10)
width_sat = 10*np.ones_like(wav_sat)

refl_band = band_integration.pixel_int(
   d=reflectance_HYP,
   x=wavelength_HYP,
   x_pixel=wav_sat,
   width_pixel=width_sat,
   band_shape="triangle"
)

print(refl_band.shape)

Finally, it is also possible to manually provide an array which provides the SRF for each band directly.
Here for convenience, we are generating these using the fd.f_triangle function (which is essentially the same as what happens in the step above), but any array could be provided.

In [ ]:
from matheo.utils import function_def as fd

wav_SRF = np.arange(390,1610,0.1)
r_SRF = np.array([fd.f_triangle(wav_SRF, sat_wav_i, 10) for sat_wav_i in wav_sat])
refl_band2 = band_integration.band_int(reflectance_HYP, wavelength_HYP, r_SRF, wav_SRF)

Next, to illustrate results we plot each of the band integrated spectra and the HYPERNETS spectrum they were derived from.
The land HYPERNETS sequences contain many different series (different viewing geometries). Each series was band integrated using the methods above.
For clarity, we here only plot a single series (with randomly selected index 30).

In [ ]:
import matplotlib.pyplot as plt

plt.plot(wavelength_HYP,reflectance_HYP[:,30],label="HYPERNETS reflectance for index 30")
plt.fill_between(wavelength_HYP,reflectance_HYP[:,30]-u_reflectance_HYP[:,30],reflectance_HYP[:,30]+u_reflectance_HYP[:,30], alpha=0.3, label="HYPERNETS reflectance uncertainty")
plt.plot(wav_sat,refl_band[:,30],label="pixel_int with 10nm sampling and 10 nm width")
plt.plot(wav_sat,refl_band2[:,30],label="band_int with 10nm sampling and 10 nm width")
plt.plot(band_centres_S2,refl_S2[:,30],"s",label="spectral_band_int_sensor with S2A SRF")
plt.legend()
plt.ylim([0,0.6])
plt.show()